In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
import tensorflow as tf
import binascii
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation

In [3]:
csv_data = pd.read_csv('normal1.csv')

In [4]:
csv_data

,length,timestamp,eth.src,eth.dst,eth.type,ip.src,ip.dst,ip.version,ip.proto,ip.len,...,load_13,load_14,load_15,load_16,load_17,load_18,load_19,protocol,protocol.sport,protocol.dport
0,144,1.647531e+09,6a:77:c0:85:72:c3,ce:4c:77:6b:35:fc,2048,10.244.0.15,10.244.0.14,4,6,130,...,5,4,12,4,5,4,3,TCP,52866,3306
1,144,1.647531e+09,ba:c6:5f:04:4f:62,ce:4c:77:6b:35:fc,2048,10.244.0.15,10.244.0.14,4,6,130,...,5,4,12,4,5,4,3,TCP,52866,3306
2,66,1.647531e+09,ce:4c:77:6b:35:fc,6a:77:c0:85:72:c3,2048,10.244.0.14,10.244.0.15,4,6,52,...,0,0,0,0,0,0,0,TCP,3306,52866
3,78,1.647531e+09,ce:4c:77:6b:35:fc,6a:77:c0:85:72:c3,2048,10.244.0.14,10.244.0.15,4,6,64,...,0,0,0,0,0,0,0,TCP,3306,52866
4,186,1.647531e+09,ce:4c:77:6b:35:fc,6a:77:c0:85:72:c3,2048,10.244.0.14,10.244.0.15,4,6,172,...,0,0,0,0,2,0,3,TCP,3306,52866
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140533,203,1.647532e+09,3a:e0:26:66:ba:06,ba:c6:5f:04:4f:62,2048,10.244.0.9,10.244.0.1,4,6,189,...,14,3,1,2,0,3,2,TCP,8080,50410
140534,66,1.647532e+09,ba:c6:5f:04:4f:62,3a:e0:26:66:ba:06,2048,10.244.0.1,10.244.0.9,4,6,52,...,0,0,0,0,0,0,0,TCP,50410,8080
140535,66,1.647532e+09,3a:e0:26:66:ba:06,ba:c6:5f:04:4f:62,2048,10.244.0.9,10.244.0.1,4,6,52,...,0,0,0,0,0,0,0,TCP,8080,50410
140536,66,1.647532e+09,ba:c6:5f:04:4f:62,3a:e0:26:66:ba:06,2048,10.244.0.1,10.244.0.9,4,6,52,...,0,0,0,0,0,0,0,TCP,50410,8080


In [5]:
csv_data.columns

Index(['length', 'timestamp', 'eth.src', 'eth.dst', 'eth.type', 'ip.src',
       'ip.dst', 'ip.version', 'ip.proto', 'ip.len', 'ip.ihl', 'ip.tos',
       'ip.ttl', 'load.count', 'FIN', 'SYN', 'RST', 'PSH', 'ACK', 'URG', 'ECE',
       'CWR', 'UNK', 'source_docker_bridge', 'source_dns', 'source_service',
       'source_pod', 'source_external', 'destination_docker_bridge',
       'destination_dns', 'destination_service', 'destination_pod',
       'destination_external', 'load_0', 'load_1', 'load_2', 'load_3',
       'load_4', 'load_5', 'load_6', 'load_7', 'load_8', 'load_9', 'load_10',
       'load_11', 'load_12', 'load_13', 'load_14', 'load_15', 'load_16',
       'load_17', 'load_18', 'load_19', 'protocol', 'protocol.sport',
       'protocol.dport'],
      dtype='object')

In [6]:
import custom_preprocessing
final_df = custom_preprocessing.process_df(csv_data)

In [7]:
csv_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140538 entries, 0 to 140537
Data columns (total 56 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   length                     140538 non-null  int64  
 1   timestamp                  140538 non-null  float64
 2   eth.src                    140538 non-null  object 
 3   eth.dst                    140538 non-null  object 
 4   eth.type                   140538 non-null  int64  
 5   ip.src                     140538 non-null  object 
 6   ip.dst                     140538 non-null  object 
 7   ip.version                 140538 non-null  int64  
 8   ip.proto                   140538 non-null  int64  
 9   ip.len                     140538 non-null  int64  
 10  ip.ihl                     140538 non-null  int64  
 11  ip.tos                     140538 non-null  int64  
 12  ip.ttl                     140538 non-null  int64  
 13  load.count                 14

In [8]:
list_of_cols_straight = ['length', 'eth.type', 'ip.version', 'ip.proto', 'ip.len', 'ip.ihl', 'ip.tos',
       'ip.ttl' ,'load.count', 'FIN', 'SYN', 'RST', 'PSH', 'ACK',
       'URG', 'ECE', 'CWR', 'UNK', 'source_docker_bridge', 'source_dns',
       'source_service', 'source_pod', 'source_external',
       'destination_docker_bridge', 'destination_dns', 'destination_service',
       'destination_pod', 'destination_external', 'load_0', 'load_1', 'load_2',
       'load_3', 'load_4', 'load_5', 'load_6', 'load_7', 'load_8', 'load_9',
       'load_10', 'load_11', 'load_12', 'load_13', 'load_14', 'load_15',
       'load_16', 'load_17', 'load_18', 'load_19',
       'protocol.sport', 'protocol.dport']

for col in list_of_cols_straight:
    final_df[str(col)] = list(csv_data[str(col)]) 

In [9]:
dummy_protocol = pd.get_dummies(csv_data['protocol'])
final_df['TCP'] = list(dummy_protocol['TCP'])
final_df['UDP'] = list(dummy_protocol['UDP'])

In [10]:
final_df = final_df.drop(['protocol'], axis = 1)
# 

# md = csv_data['load'][0]

# final_df[[f'load_{x}' for x in range(20)]] = None

KeyError: "['protocol'] not found in axis"

In [11]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140538 entries, 0 to 140537
Data columns (total 72 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   octet_1_eth_dst            140538 non-null  int64 
 1   octet_2_eth_dst            140538 non-null  int64 
 2   octet_3_eth_dst            140538 non-null  int64 
 3   octet_4_eth_dst            140538 non-null  int64 
 4   octet_5_eth_dst            140538 non-null  int64 
 5   octet_6_eth_dst            140538 non-null  int64 
 6   octet_1_eth_src            140538 non-null  int64 
 7   octet_2_eth_src            140538 non-null  int64 
 8   octet_3_eth_src            140538 non-null  int64 
 9   octet_4_eth_src            140538 non-null  int64 
 10  octet_5_eth_src            140538 non-null  int64 
 11  octet_6_eth_src            140538 non-null  int64 
 12  octet_1_ip_src             140538 non-null  object
 13  octet_2_ip_src             140538 non-null  

In [ ]:
# hex_lambda = lambda x: int(x,16)

# clean_df = pd.DataFrame()

# clean_df[[f'load_{x}' for x in range(20)]] = None

# for index, row in csv_data['load'].iteritems():
#     if row != str(0):
#         if len(row)!=20:
#             row = row + '0'*(20-len(row))
#         clean_df.loc[index] = [hex_lambda(ch) for ch in row]
#     else:
#         clean_df.loc[index] = [0]*20

In [ ]:
# new_df = pd.concat([final_df, clean_df], axis = 1)

In [ ]:
# csv_data.columns

In [ ]:
new_df

In [12]:
final_df.columns

Index(['octet_1_eth_dst', 'octet_2_eth_dst', 'octet_3_eth_dst',
       'octet_4_eth_dst', 'octet_5_eth_dst', 'octet_6_eth_dst',
       'octet_1_eth_src', 'octet_2_eth_src', 'octet_3_eth_src',
       'octet_4_eth_src', 'octet_5_eth_src', 'octet_6_eth_src',
       'octet_1_ip_src', 'octet_2_ip_src', 'octet_3_ip_src', 'octet_4_ip_src',
       'octet_1_ip_dst', 'octet_2_ip_dst', 'octet_3_ip_dst', 'octet_4_ip_dst',
       'length', 'eth.type', 'ip.version', 'ip.proto', 'ip.len', 'ip.ihl',
       'ip.tos', 'ip.ttl', 'load.count', 'FIN', 'SYN', 'RST', 'PSH', 'ACK',
       'URG', 'ECE', 'CWR', 'UNK', 'source_docker_bridge', 'source_dns',
       'source_service', 'source_pod', 'source_external',
       'destination_docker_bridge', 'destination_dns', 'destination_service',
       'destination_pod', 'destination_external', 'load_0', 'load_1', 'load_2',
       'load_3', 'load_4', 'load_5', 'load_6', 'load_7', 'load_8', 'load_9',
       'load_10', 'load_11', 'load_12', 'load_13', 'load_14', 'load_1

In [13]:
from sklearn.preprocessing import StandardScaler
columns_not_to_scale = ['FIN', 'SYN', 'RST', 'PSH', 'ACK', 'URG', 'ECE', 'CWR', 'UNK',
       'source_docker_bridge', 'source_dns', 'source_service', 'source_pod',
       'source_external', 'destination_docker_bridge', 'destination_dns',
       'destination_service', 'destination_pod', 'destination_external', 'TCP',
       'UDP']

for col in final_df.columns:
    if col not in columns_not_to_scale:
        scale = StandardScaler().fit(final_df[[col]])
        final_df[col] = scale.transform(final_df[[col]])

In [14]:
final_df

,octet_1_eth_dst,octet_2_eth_dst,octet_3_eth_dst,octet_4_eth_dst,octet_5_eth_dst,octet_6_eth_dst,octet_1_eth_src,octet_2_eth_src,octet_3_eth_src,octet_4_eth_src,...,load_14,load_15,load_16,load_17,load_18,load_19,protocol.sport,protocol.dport,TCP,UDP
0,0.999558,-0.798079,0.085577,0.396893,-1.027218,1.371663,-0.752915,-0.288377,1.992587,1.069766,...,0.667342,2.005733,0.618697,0.791433,0.664259,0.313269,1.026406,-0.972130,1,0
1,0.999558,-0.798079,0.085577,0.396893,-1.027218,1.371663,0.553287,0.945509,-0.263293,-1.085399,...,0.667342,2.005733,0.618697,0.791433,0.664259,0.313269,1.026406,-0.972130,1,0
2,-0.581271,-0.090221,1.483294,0.847824,0.285020,0.594669,0.879837,-0.959987,0.294863,0.635392,...,-0.512638,-0.565434,-0.524846,-0.565851,-0.515196,-0.538543,-1.080588,1.112464,1,0
3,-0.581271,-0.090221,1.483294,0.847824,0.285020,0.594669,0.879837,-0.959987,0.294863,0.635392,...,-0.512638,-0.565434,-0.524846,-0.565851,-0.515196,-0.538543,-1.080588,1.112464,1,0
4,-0.581271,-0.090221,1.483294,0.847824,0.285020,0.594669,0.879837,-0.959987,0.294863,0.635392,...,-0.512638,-0.565434,-0.524846,-0.022937,-0.515196,0.313269,-1.080588,1.112464,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140533,0.683392,1.210262,-0.373946,-1.389487,-0.467903,-0.727583,-1.536636,1.351599,-1.588913,0.551858,...,0.372347,-0.351170,0.046925,-0.565851,0.369395,0.029332,-0.877626,1.009160,1,0
140534,-1.340069,1.638269,-1.465314,0.310175,1.833891,-1.981678,0.553287,0.945509,-0.263293,-1.085399,...,-0.512638,-0.565434,-0.524846,-0.565851,-0.515196,-0.538543,0.921991,-0.771326,1,0
140535,0.683392,1.210262,-0.373946,-1.389487,-0.467903,-0.727583,-1.536636,1.351599,-1.588913,0.551858,...,-0.512638,-0.565434,-0.524846,-0.565851,-0.515196,-0.538543,-0.877626,1.009160,1,0
140536,-1.340069,1.638269,-1.465314,0.310175,1.833891,-1.981678,0.553287,0.945509,-0.263293,-1.085399,...,-0.512638,-0.565434,-0.524846,-0.565851,-0.515196,-0.538543,0.921991,-0.771326,1,0


In [15]:
final_df = final_df.astype('float64')

In [16]:
from sklearn.model_selection import train_test_split

x_normal = final_df.values

x_normal_train, x_normal_test = train_test_split(
    x_normal, test_size=0.25, random_state=42)

In [17]:
x_normal.shape

(140538, 72)

In [18]:
#training on the normal data
#autoencoder 
model = Sequential()
model.add(Dense(40, input_dim=x_normal.shape[1], activation='relu'))
model.add(Dense(4, activation='relu')) # size to compress to
model.add(Dense(40, activation='relu'))
model.add(Dense(x_normal.shape[1])) # Multiple output neurons
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x_normal_train,x_normal_train,verbose=1,epochs=100)

Metal device set to: Apple M1 Pro
Epoch 1/100


2022-03-17 17:17:49.834709: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-17 17:17:49.835123: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-03-17 17:17:49.951605: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-03-17 17:17:50.088021: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


3294/3294 [==============================] - 13s 4ms/step - loss: 0.2446
Epoch 2/100
3294/3294 [==============================] - 13s 4ms/step - loss: 0.1616
Epoch 3/100
3294/3294 [==============================] - 13s 4ms/step - loss: 0.1463
Epoch 4/100
3294/3294 [==============================] - 13s 4ms/step - loss: 0.1388
Epoch 5/100
3294/3294 [==============================] - 13s 4ms/step - loss: 0.1338
Epoch 6/100
3294/3294 [==============================] - 13s 4ms/step - loss: 0.1300
Epoch 7/100
3294/3294 [==============================] - 13s 4ms/step - loss: 0.1267
Epoch 8/100
3294/3294 [==============================] - 13s 4ms/step - loss: 0.1242
Epoch 9/100
3294/3294 [==============================] - 13s 4ms/step - loss: 0.1223
Epoch 10/100
3294/3294 [==============================] - 13s 4ms/step - loss: 0.1205
Epoch 11/100
3294/3294 [==============================] - 13s 4ms/step - loss: 0.1191
Epoch 12/100
3294/3294 [==============================] - 13s 4ms/step - lo

3294/3294 [==============================] - 13s 4ms/step - loss: 0.1002
Epoch 97/100
3294/3294 [==============================] - 13s 4ms/step - loss: 0.1001
Epoch 98/100
3294/3294 [==============================] - 13s 4ms/step - loss: 0.1000
Epoch 99/100
3294/3294 [==============================] - 15s 4ms/step - loss: 0.1001
Epoch 100/100
3294/3294 [==============================] - 14s 4ms/step - loss: 0.1001


In [19]:
model.save("model_1")

INFO:tensorflow:Assets written to: model_1/assets


2022-03-17 17:39:59.795723: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


In [21]:
pred = model.predict(x_normal_test)
score1 = np.sqrt(metrics.mean_squared_error(pred,x_normal_test))

2022-03-17 17:41:35.880020: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [22]:
score1

0.3161374555226119

In [24]:
pred2 = model.predict(x_normal_train)
score2 = np.sqrt(metrics.mean_squared_error(pred2,x_normal_train))

In [25]:
score2

0.314486677761485

In [26]:
attack_csv = pd.read_csv('attack1.csv')

In [31]:
new_csv = attack_csv[attack_csv['ip.dst'] == '89.100.107.148']

In [33]:
new_df = custom_preprocessing.process_df(attack_csv)

In [34]:
list_of_cols_straight = ['length', 'eth.type', 'ip.version', 'ip.proto', 'ip.len', 'ip.ihl', 'ip.tos',
       'ip.ttl' ,'load.count', 'FIN', 'SYN', 'RST', 'PSH', 'ACK',
       'URG', 'ECE', 'CWR', 'UNK', 'source_docker_bridge', 'source_dns',
       'source_service', 'source_pod', 'source_external',
       'destination_docker_bridge', 'destination_dns', 'destination_service',
       'destination_pod', 'destination_external', 'load_0', 'load_1', 'load_2',
       'load_3', 'load_4', 'load_5', 'load_6', 'load_7', 'load_8', 'load_9',
       'load_10', 'load_11', 'load_12', 'load_13', 'load_14', 'load_15',
       'load_16', 'load_17', 'load_18', 'load_19',
       'protocol.sport', 'protocol.dport']

for col in list_of_cols_straight:
    new_df[str(col)] = list(attack_csv[str(col)]) 

In [35]:
dummy_protocol = pd.get_dummies(attack_csv['protocol'])
new_df['TCP'] = list(dummy_protocol['TCP'])
new_df['UDP'] = list(dummy_protocol['UDP'])

In [36]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1070554 entries, 0 to 1070553
Data columns (total 72 columns):
 #   Column                     Non-Null Count    Dtype 
---  ------                     --------------    ----- 
 0   octet_1_eth_dst            1070554 non-null  int64 
 1   octet_2_eth_dst            1070554 non-null  int64 
 2   octet_3_eth_dst            1070554 non-null  int64 
 3   octet_4_eth_dst            1070554 non-null  int64 
 4   octet_5_eth_dst            1070554 non-null  int64 
 5   octet_6_eth_dst            1070554 non-null  int64 
 6   octet_1_eth_src            1070554 non-null  int64 
 7   octet_2_eth_src            1070554 non-null  int64 
 8   octet_3_eth_src            1070554 non-null  int64 
 9   octet_4_eth_src            1070554 non-null  int64 
 10  octet_5_eth_src            1070554 non-null  int64 
 11  octet_6_eth_src            1070554 non-null  int64 
 12  octet_1_ip_src             1070554 non-null  object
 13  octet_2_ip_src             

In [37]:
from sklearn.preprocessing import StandardScaler
columns_not_to_scale = ['FIN', 'SYN', 'RST', 'PSH', 'ACK', 'URG', 'ECE', 'CWR', 'UNK',
       'source_docker_bridge', 'source_dns', 'source_service', 'source_pod',
       'source_external', 'destination_docker_bridge', 'destination_dns',
       'destination_service', 'destination_pod', 'destination_external', 'TCP',
       'UDP']

for col in new_df.columns:
    if col not in columns_not_to_scale:
        scale = StandardScaler().fit(new_df[[col]])
        new_df[col] = scale.transform(new_df[[col]])

In [38]:
new_df

,octet_1_eth_dst,octet_2_eth_dst,octet_3_eth_dst,octet_4_eth_dst,octet_5_eth_dst,octet_6_eth_dst,octet_1_eth_src,octet_2_eth_src,octet_3_eth_src,octet_4_eth_src,...,load_14,load_15,load_16,load_17,load_18,load_19,protocol.sport,protocol.dport,TCP,UDP
0,-1.920957,1.939359,-1.254695,-0.792707,1.308619,-0.874206,0.432497,1.286989,-0.786805,-1.394521,...,4.489033,3.197718,2.849936,3.690354,1.876597,2.290384,-0.711639,1.467484,1,0
1,0.608604,2.344926,-1.140939,-2.876077,-0.260861,-1.791539,-2.364256,0.994221,-0.905049,-0.011451,...,3.126219,2.950950,5.730382,1.463661,0.924988,0.579754,0.570995,-0.634366,1,0
2,-1.920957,1.939359,-1.254695,-0.792707,1.308619,-0.874206,0.432497,1.286989,-0.786805,-1.394521,...,2.217677,0.730033,3.330010,4.326553,4.731424,4.343141,-0.771593,0.701942,1,0
3,0.608604,2.344926,-1.140939,-2.876077,-0.260861,-1.791539,-2.364256,0.994221,-0.905049,-0.011451,...,-0.507951,-0.503810,-0.510585,-0.444934,-0.502425,-0.446624,0.570995,-0.634366,1,0
4,0.608604,2.344926,-1.140939,-2.876077,-0.260861,-1.791539,-2.364256,0.994221,-0.905049,-0.011451,...,-0.507951,-0.503810,-0.510585,-0.444934,-0.502425,-0.446624,1.340134,-0.574692,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1070549,-1.844303,3.003971,-2.437764,-0.189626,2.585484,-3.354403,0.432497,1.286989,-0.786805,-1.394521,...,-0.507951,-0.503810,-0.510585,-0.444934,-0.502425,-0.446624,-1.200721,1.605593,0,1
1070550,-2.534183,-1.457257,-3.029298,-1.176485,-0.899294,-2.488033,-2.279506,1.762736,-2.134783,0.388911,...,-0.507951,-0.503810,-0.510585,-0.444934,-0.502425,-0.446624,-1.200721,1.315722,0,1
1070551,-2.534183,-1.457257,-3.029298,-1.176485,-0.899294,-2.488033,0.432497,1.286989,-0.786805,-1.394521,...,-0.507951,-0.503810,-0.510585,-0.444934,-0.502425,-0.446624,-1.182015,1.184583,1,0
1070552,0.608604,2.344926,-1.140939,-2.876077,-0.260861,-1.791539,-3.042256,-1.457706,-2.749651,-0.266227,...,-0.507951,-0.503810,-0.510585,-0.444934,-0.502425,-0.446624,1.055904,-1.042868,1,0


In [40]:
x_attack = new_df.values

In [41]:
attack_pred = model.predict(x_attack)

In [44]:
score_list = []
for index, x in enumerate(attack_pred):
    score_list.append(np.sqrt(metrics.mean_squared_error(attack_pred[index],x_attack[index])))

In [45]:
attack_csv['score'] = score_list

In [47]:
score_test = attack_csv[attack_csv['ip.dst'] == '89.100.107.148']

In [50]:
score_test['score']

1214       0.553545
1319       0.554121
1322       0.553746
1498       0.553751
1555       0.553752
             ...   
1061736    0.569236
1061751    0.540156
1061752    0.569116
1061764    0.539995
1061765    0.569294
Name: score, Length: 41824, dtype: float64

In [52]:
score_test['score'].mean()

0.5686491358968594

In [54]:
attack_csv['score'].mean()

0.49114924821364203